In [3]:
!pip install -U hopsworks

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.4/646.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 22.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 29.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 33.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.8 MB/s eta 0:00:00
  Created wheel for avro: filename=avro-1.11.3-py2.py3-none-any.whl size=123913 sha

In [13]:
!pip install pyarrow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import hopsworks
import pandas as pd

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
trucks_df= pd.read_csv("trucks_table.csv")
trucks_df.head(10)

,truck_id,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,42302347,10,3000.0,17,gas
1,27867488,14,10000.0,22,diesel
2,13927774,8,10000.0,19,gas
3,69577118,8,20000.0,19,gas
4,28650047,10,4000.0,21,diesel
5,29303248,9,4000.0,15,gas
6,76304377,8,3000.0,20,diesel
7,21093078,8,NaN,14,gas
8,41713749,9,NaN,22,gas
9,14730346,11,15000.0,20,NaN


In [6]:
trucks_df.isna().sum()

truck_id                 0
truck_age                0
load_capacity_pounds    57
mileage_mpg              0
fuel_type               40
dtype: int64

In [7]:
trucks_df['fuel_type']=trucks_df['fuel_type'].fillna('Unknown')

In [8]:
trucks_df.isna().sum()

truck_id                 0
truck_age                0
load_capacity_pounds    57
mileage_mpg              0
fuel_type                0
dtype: int64

In [9]:
trucks_df['event_time']=pd.to_datetime("2023-08-23")

In [10]:
trucks_df=trucks_df.sort_values(['event_time','truck_id'])

In [11]:
trucks_df[trucks_df.duplicated(subset=['truck_id'])]

,truck_id,truck_age,load_capacity_pounds,mileage_mpg,fuel_type,event_time


In [2]:
api_key = "wfTwb1hSTRESvdV1.FP4GjRqsGb2mGEcKUq5zZTfSPPAuicaUL8QZsWvr7ZpjhqA86CPa4H38MvhS439H"
project_name = "shankarg"

In [4]:
project = hopsworks.login(api_key_value=api_key,project=project_name)
fs=project.get_feature_store()
fg = fs.get_or_create_feature_group(
        name="trucks_df", 
        version=1, 
        description="Details of the Truck", 
        event_time = "event_time",
        primary_key=['truck_id'], 
        online_enabled=False)

2025-03-02 13:19:15,230 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 13:19:15,232 INFO: Initializing external client
2025-03-02 13:19:15,233 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 13:19:17,094 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/779413


In [12]:
fg.insert(trucks_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/779413/fs/775236/fg/1403296


Uploading Dataframe: 100.00% |██████████| Rows 1300/1300 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: trucks_df_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/779413/jobs/named/trucks_df_1_offline_fg_materialization/executions


(Job('trucks_df_1_offline_fg_materialization', 'SPARK'), None)

In [13]:
drivers_df= pd.read_csv("drivers_table.csv")
drivers_df.head(10)

,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph
0,d9f30553-6,Daniel Marks,male,47,5,proactive,7,42302347,62.22
1,82de7bb8-2,Clifford Carr,male,47,14,proactive,4,27867488,60.89
2,7e789842-4,Terry Faulkner MD,male,41,9,conservative,2,13927774,53.67
3,b2555587-8,Brendan Jacobs,male,44,10,proactive,2,69577118,59.82
4,b2e58421-d,Vincent Davis,male,41,10,proactive,7,28650047,62.65
5,6ef125cc-7,Alexander Wheeler,male,49,10,proactive,4,29303248,57.66
6,e2796d38-5,Frederick Perry,male,54,17,conservative,9,76304377,40.79
7,86b61bd1-8,Todd Garrison,male,49,5,NaN,4,21093078,61.70
8,24d78a24-e,John Anthony,male,48,0,conservative,3,41713749,42.15
9,09665f38-8,Robert Russell DDS,male,40,6,conservative,2,14730346,46.49


In [14]:
drivers_df.isna().sum()

driver_id             0
name                  0
gender               23
age                   0
experience            0
driving_style        52
ratings               0
vehicle_no            0
average_speed_mph     0
dtype: int64

In [15]:
drivers_df['gender']=drivers_df['gender'].fillna('Unknown')

In [16]:
 drivers_df.isna().sum()

driver_id             0
name                  0
gender                0
age                   0
experience            0
driving_style        52
ratings               0
vehicle_no            0
average_speed_mph     0
dtype: int64

In [17]:
drivers_df['driving_style'].value_counts()

driving_style
proactive       648
conservative    600
Name: count, dtype: int64

In [18]:
drivers_df['driving_style']=drivers_df['driving_style'].fillna('Unknown')

In [19]:
drivers_df.isna().sum()

driver_id            0
name                 0
gender               0
age                  0
experience           0
driving_style        0
ratings              0
vehicle_no           0
average_speed_mph    0
dtype: int64

In [20]:
drivers_df['event_time']=pd.to_datetime("2023-08-23")

In [21]:
drivers_df= drivers_df.sort_values(['event_time','driver_id'])

In [22]:
driver_fg = fs.get_or_create_feature_group(
        name="drivers_df", 
        version=1, 
        description="Details of the Truck", 
        event_time = "event_time",
        primary_key=['driver_id'], 
        online_enabled=False)

In [23]:
driver_fg.insert(drivers_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/779413/fs/775236/fg/1403298


Uploading Dataframe: 100.00% |██████████| Rows 1300/1300 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: drivers_df_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/779413/jobs/named/drivers_df_1_offline_fg_materialization/executions


(Job('drivers_df_1_offline_fg_materialization', 'SPARK'), None)